In [ ]:
from IPython.display import display, Markdown  # 在jupyter显示信息的工具

In [4]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
# from langchain.python import PythonREPL

from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

In [42]:
# 参数temperature设置为0.0，从而减少生成答案的随机性。
llm = ChatOllama(model="qwen2.5", temperature=0.0)

tools = load_tools(["llm-math", "wikipedia"], llm=llm)

In [43]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [7]:
# 初始化代理
agent = initialize_agent(
    tools,  # 第二步加载的工具
    llm,  # 第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # 代理类型
    handle_parsing_errors=True,  # 处理解析错误
    verbose=True,  # 输出中间步骤
)

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_99582/158468433.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
agent("计算300的25%")

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_99582/4002985145.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent("计算300的25%")




> Entering new AgentExecutor chain...
Question: 计算300的25%
Thought: 这是一个数学问题，需要计算300的25%。
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```
Observation: Answer: 75.0
Thought:我得到了正确的结果。

Final Answer: 75

> Finished chain.


{'input': '计算300的25%', 'output': '75'}

In [10]:
question = "Tom M. Mitchell是一位美国计算机科学家，\
也是卡内基梅隆大学（CMU）的创始人大学教授。\
他写了哪本书呢？"

agent(question)



> Entering new AgentExecutor chain...
Thought: 这个问题需要查询Tom M. Mitchell写过的书，应该使用wikipedia工具来查找相关信息。
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtai

{'input': 'Tom M. Mitchell是一位美国计算机科学家，也是卡内基梅隆大学（CMU）的创始人大学教授。他写了哪本书呢？',
 'output': 'Tom M. Mitchell撰写的书籍是《Machine Learning》。'}

In [35]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,  # 使用前面一节已经加载的大语言模型
    tool=PythonREPLTool(),  # 使用Python交互式环境工具 REPLTool
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # 代理类型
    handle_parsing_errors=True,  # 处理解析错误
    verbose=True,  # 输出中间步骤
)
customer_list = [
    "小明",
    "小黄",
    "小红",
    "小蓝",
    "小橘",
    "小绿",
]

agent.run(
    f"将使用pinyin拼音库这些客户名字转换为拼音，并打印输出列表: {customer_list}。"
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "将使用pinyin拼音库这些客户名字转换为拼音，并打印输出列表: ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']。"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "将使用pinyin拼音库这些客户名字转换为拼音，并打印输出列表: ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']。",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I

"使用pypinyin库将给定的客户名字转换为拼音后，列表如下：[['xiao', 'ming'], ['xiao', 'huang'], ['xiao', 'hong'], ['xiao', 'lan'], ['xiao', 'ju'], ['xiao', 'lv']]。"

In [20]:
import langchain
langchain.debug = True
agent.run(f"使用pinyin拼音库将这些客户名字转换为拼音，并打印输出列表: {customer_list}")
langchain.debug = False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "使用pinyin拼音库将这些客户名字转换为拼音，并打印输出列表: ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "使用pinyin拼音库将这些客户名字转换为拼音，并打印输出列表: ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I d

In [46]:
# 导入tool函数装饰器
from langchain.agents import tool
from datetime import date


@tool
def time(text: str) -> str:
    """
    返回今天的日期，用于任何需要知道今天日期的问题。\
    输入应该总是一个空字符串，\
    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。
    """
    return str(date.today())

# tools = load_tools([time, "llm-math", "wikipedia"], llm=llm)

# 初始化代理
agent = initialize_agent(
    tools=[time,],  # 将刚刚创建的时间工具加入代理
    llm=llm,  # 初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # 代理类型
    handle_parsing_errors=True,  # 处理解析错误
    verbose=True,  # 输出中间步骤
)

langchain.debug = True
# 使用代理询问今天的日期.
# 注: 代理有时候可能会出错（该功能正在开发中）。如果出现错误，请尝试再次运行它。
agent("今天的日期是？")
# agent("今天的日期是？ 往后两天是几号？ 有哪个名人是当天出生的？列举一个就行")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "今天的日期是？"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "今天的日期是？",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\ntime: 返回今天的日期，用于任何需要知道今天日期的问题。    输入应该总是一个空字符串，    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the \"action\" field are: time\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{\n  \"

{'input': '今天的日期是？', 'output': '今天的日期是2025年2月26日。'}

In [47]:
langchain.debug = False
agent("今天的日期是？")



> Entering new AgentExecutor chain...
Thought: 这个问题需要知道今天的日期，应该使用time工具来获取。
Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation: 2025-02-26
Thought:I now know the final answer
Final Answer: 今天的日期是2025年2月26日。

> Finished chain.


{'input': '今天的日期是？', 'output': '今天的日期是2025年2月26日。'}